**1970. Last Day Where You Can Still Cross**

**Hard**

**Companies**: Meta Google

There is a 1-based binary matrix where 0 represents land and 1 represents water. You are given integers row and col representing the number of rows and columns in the matrix, respectively.

Initially on day 0, the entire matrix is land. However, each day a new cell becomes flooded with water. You are given a 1-based 2D array cells, where cells[i] = [ri, ci] represents that on the ith day, the cell on the rith row and cith column (1-based coordinates) will be covered with water (i.e., changed to 1).

You want to find the last day that it is possible to walk from the top to the bottom by only walking on land cells. You can start from any cell in the top row and end at any cell in the bottom row. You can only travel in the four cardinal directions (left, right, up, and down).

Return the last day where it is possible to walk from the top to the bottom by only walking on land cells.

**Example 1:**

```python
Input: row = 2, col = 2, cells = [[1,1],[2,1],[1,2],[2,2]]
Output: 2
```

**Explanation:** The above image depicts how the matrix changes each day starting from day 0.
The last day where it is possible to cross from top to bottom is on day 2.

**Example 2:**

```python
Input: row = 2, col = 2, cells = [[1,1],[1,2],[2,1],[2,2]]
Output: 1
```

**Explanation:** The above image depicts how the matrix changes each day starting from day 0.
The last day where it is possible to cross from top to bottom is on day 1.

**Example 3:**

```python
Input: row = 3, col = 3, cells = [[1,2],[2,1],[3,3],[2,2],[1,1],[1,3],[2,3],[3,2],[3,1]]
Output: 3
```

**Explanation:** The above image depicts how the matrix changes each day starting from day 0.
The last day where it is possible to cross from top to bottom is on day 3.

**Constraints:**

- 2 <= row, col <= 2 \* 104
- 4 <= row _ col <= 2 _ 104
- cells.length == row \* col
- 1 <= ri <= row
- 1 <= ci <= col
- All the values of cells are unique.


In [ ]:
from typing import List
from collections import deque


class Solution:
    def latestDayToCross(self, row: int, col: int, cells: List[List[int]]) -> int:
        """
        Binary Search + BFS approach.

        Algorithm:
        1. Use binary search on days (0 → row*col).
        2. For a given day mid:
           - Mark first 'mid' cells as water.
           - Run BFS from all land cells in the top row.
           - If any path reaches the bottom row → crossing possible.
        3. If crossing possible:
           - Move right (try later days).
        4. Else:
           - Move left (try earlier days).
        5. Return the maximum valid day.

        Time Complexity:
        O((row * col) * log(row * col))

        Space Complexity:
        O(row * col)
        """

        def can_cross(day: int) -> bool:
            # Build grid: 0 = land, 1 = water
            grid = [[0] * col for _ in range(row)]
            for i in range(day):
                r, c = cells[i]
                grid[r - 1][c - 1] = 1

            q = deque()
            visited = [[False] * col for _ in range(row)]

            # Start BFS from all land cells in top row
            for j in range(col):
                if grid[0][j] == 0:
                    q.append((0, j))
                    visited[0][j] = True

            directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

            while q:
                x, y = q.popleft()
                if x == row - 1:
                    return True  # reached bottom

                for dx, dy in directions:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < row and 0 <= ny < col:
                        if not visited[nx][ny] and grid[nx][ny] == 0:
                            visited[nx][ny] = True
                            q.append((nx, ny))

            return False

        left, right = 0, row * col
        answer = 0

        while left <= right:
            mid = (left + right) // 2
            if can_cross(mid):
                answer = mid
                left = mid + 1
            else:
                right = mid - 1

        return answer


In [ ]:
from typing import List


class Solution:
    def latestDayToCross(self, row: int, col: int, cells: List[List[int]]) -> int:
        """
        Binary Search + DFS approach.

        Algorithm:
        ----------------
        1. Observe monotonicity:
           - If crossing is possible on day d,
             it is possible on all days < d.
           - If crossing is not possible on day d,
             it is impossible on all days > d.

        2. Use Binary Search on days:
           - Search range: [0, row * col]
           - For each mid day:
             a) Flood the first `mid` cells
             b) Check if a path exists from top to bottom using DFS

        3. DFS Check:
           - Start DFS from all land cells in the top row
           - Move only through land cells (0)
           - If any DFS reaches the bottom row → crossing possible

        4. Binary Search Decision:
           - If crossing possible → try later days
           - Else → try earlier days

        Time Complexity:
        ----------------
        O((row * col) * log(row * col))
        - Each DFS takes O(row * col)
        - Binary search runs log(row * col) times

        Space Complexity:
        -----------------
        O(row * col)
        - Grid + visited array + DFS stack
        """

        # Directions for 4-way movement (up, down, left, right)
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        def can_cross(day: int) -> bool:
            """
            Checks if crossing is possible on a given day using DFS.
            """

            # Step 1: Build grid for this day
            # 0 = land, 1 = water
            grid = [[0] * col for _ in range(row)]
            for i in range(day):
                r, c = cells[i]
                grid[r - 1][c - 1] = 1

            visited = [[False] * col for _ in range(row)]

            # Step 2: DFS from every land cell in the top row
            for j in range(col):
                if grid[0][j] == 0 and not visited[0][j]:
                    stack = [(0, j)]
                    visited[0][j] = True

                    # Iterative DFS
                    while stack:
                        x, y = stack.pop()

                        # If bottom row reached → crossing possible
                        if x == row - 1:
                            return True

                        # Explore neighbors
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < row and 0 <= ny < col:
                                if not visited[nx][ny] and grid[nx][ny] == 0:
                                    visited[nx][ny] = True
                                    stack.append((nx, ny))

            # No path found
            return False

        # Step 3: Binary search on days
        left, right = 0, row * col
        answer = 0

        while left <= right:
            mid = (left + right) // 2

            if can_cross(mid):
                answer = mid          # Valid day found
                left = mid + 1        # Try later days
            else:
                right = mid - 1       # Try earlier days

        return answer


In [ ]:
class Solution:
    def latestDayToCross(self, row: int, col: int, cells: List[List[int]]) -> int:
        """
        Reverse Union-Find approach.

        Algorithm:
        1. Initially, the grid is all water.
        2. Process days in reverse (from last to first).
        3. Each day:
           - Turn the cell into land.
           - Union it with adjacent land cells.
        4. Maintain two virtual nodes:
           - TOP (connected to top row)
           - BOTTOM (connected to bottom row)
        5. When TOP and BOTTOM become connected:
           - Return the current day index.

        Time Complexity:
        O(row * col * α(n)) ~ O(row * col)

        Space Complexity:
        O(row * col)
        """

        parent = {}
        rank = {}

        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]

        def union(a, b):
            ra, rb = find(a), find(b)
            if ra != rb:
                parent[rb] = ra

        def index(r, c):
            return r * col + c

        TOP = row * col
        BOTTOM = row * col + 1

        for i in range(row * col + 2):
            parent[i] = i

        grid = [[1] * col for _ in range(row)]
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

        for day in range(len(cells) - 1, -1, -1):
            r, c = cells[day]
            r -= 1
            c -= 1
            grid[r][c] = 0
            curr = index(r, c)

            # Connect to virtual top/bottom
            if r == 0:
                union(curr, TOP)
            if r == row - 1:
                union(curr, BOTTOM)

            # Union adjacent land cells
            for dx, dy in directions:
                nr, nc = r + dx, c + dy
                if 0 <= nr < row and 0 <= nc < col and grid[nr][nc] == 0:
                    union(curr, index(nr, nc))

            # Check connectivity
            if find(TOP) == find(BOTTOM):
                return day

        return 0
